In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.initializers import *
from keras.optimizers import * 

In [ ]:
batch_size = 64  
epochs = 100  
latent_dim = 256  
num_samples = 10000  
# set the data_path accordingly
data_path = "spoc-train.txt" 

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    ############### A ###############
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    ############### B ###############
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
print(input_characters)
print(target_characters)

{'k', 'Y', 'T', 'P', 'g', 'o', 'O', 'S', '?', 'w', 'x', 'p', '"', '6', 'H', '2', 'N', '>', 'q', 'E', "'", '-', '.', 'G', '_', 'W', '+', 'd', '=', '3', 'i', 'C', '1', 'Q', 'z', '}', 'D', 'a', 'f', 'y', '*', '{', '4', 'n', '5', 'Z', 'u', '9', 'I', ')', '0', ';', '[', '&', '/', '7', 'b', 'A', '<', 'h', ']', 'L', ':', ' ', 'c', '\\', '!', 'j', 's', 'U', '8', 'F', ',', '(', 'l', '%', 'J', 'e', 'v', 'M', 'B', 'X', 'R', 'm', '|', 'V', 't', 'K', 'r'}
{'k', 'Y', 'T', 'P', 'g', 'o', 'O', 'S', '?', 'w', 'x', 'p', '"', '6', 'H', '2', 'N', '>', 'q', 'E', "'", '-', '.', 'G', '\t', '_', 'W', '+', 'd', '=', '3', 'i', 'C', '1', 'Q', '}', 'z', 'D', 'a', 'f', 'y', '*', '{', '4', 'n', '5', 'Z', 'u', '9', 'I', ')', ';', '0', '[', '&', '/', '7', 'b', 'A', '<', 'h', ']', 'L', ':', '\n', ' ', 'c', '\\', '!', 'j', 's', 'U', '8', 'F', '(', ',', 'l', '%', 'J', 'e', 'v', 'M', 'B', 'X', 'R', 'm', '|', 'V', 't', 'K', 'r'}


In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("No.of samples:", len(input_texts))
print("No.of unique input tokens:", num_encoder_tokens)
print("No.of unique output tokens:", num_decoder_tokens)
print("Maximum seq length for inputs:", max_encoder_seq_length)
print("Maximum seq length for outputs:", max_decoder_seq_length)

No.of samples: 10000
No.of unique input tokens: 89
No.of unique output tokens: 91
Maximum seq length for inputs: 165
Maximum seq length for outputs: 90


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '%': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '<': 26, '=': 27, '>': 28, '?': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, '\\': 57, ']': 58, '_': 59, 'a': 60, 'b': 61, 'c': 62, 'd': 63, 'e': 64, 'f': 65, 'g': 66, 'h': 67, 'i': 68, 'j': 69, 'k': 70, 'l': 71, 'm': 72, 'n': 73, 'o': 74, 'p': 75, 'q': 76, 'r': 77, 's': 78, 't': 79, 'u': 80, 'v': 81, 'w': 82, 'x': 83, 'y': 84, 'z': 85, '{': 86, '|': 87, '}': 88}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23

In [ ]:
import numpy as np
encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=["accuracy"])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("E2S")

Epoch 1/100
125/125 [==============================] - 6s 36ms/step - loss: 1.1216 - accuracy: 0.8081 - val_loss: 0.7206 - val_accuracy: 0.8552
Epoch 2/100
125/125 [==============================] - 4s 30ms/step - loss: 0.7349 - accuracy: 0.8527 - val_loss: 0.6295 - val_accuracy: 0.8683
Epoch 3/100
125/125 [==============================] - 4s 31ms/step - loss: 0.6646 - accuracy: 0.8609 - val_loss: 0.5770 - val_accuracy: 0.8790
Epoch 4/100
125/125 [==============================] - 4s 31ms/step - loss: 0.6217 - accuracy: 0.8684 - val_loss: 0.5443 - val_accuracy: 0.8857
Epoch 5/100
125/125 [==============================] - 4s 31ms/step - loss: 0.5833 - accuracy: 0.8738 - val_loss: 0.5137 - val_accuracy: 0.8894
Epoch 6/100
125/125 [==============================] - 4s 31ms/step - loss: 0.5553 - accuracy: 0.8790 - val_loss: 0.4930 - val_accuracy: 0.8921
Epoch 7/100
125/125 [==============================] - 4s 31ms/step - loss: 0.5333 - accuracy: 0.8818 - val_loss: 0.4722 - val_accuracy:

INFO:tensorflow:Assets written to: E2S/assets


INFO:tensorflow:Assets written to: E2S/assets


In [ ]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [ ]:
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [88]:
def inference_translater(english_input):
    
    encoder_states = encoder_model.predict(english_input)

    japanese_seq = np.zeros((1, 1, num_decoder_tokens))
    japanese_seq[0, 0, target_token_index["\t"]] = 1.
    
    stop_condition = False
    japanese_output = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([japanese_seq] + encoder_states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        japanese_output += sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == "\n" or len(japanese_output) > max_decoder_seq_length):
            stop_condition = True

        japanese_seq = np.zeros((1, 1, num_decoder_tokens))
        japanese_seq[0, 0, sampled_token_index] = 1.

        encoder_states = [h, c]

    return japanese_output

In [112]:
#i = np.random.choice(len(input_texts))
i = 114
input_seq = encoder_input_data[i:i+1]
#print(input_seq)
# translation = decode_sequence(input_seq)
# print('-')
# print('Input:', input_texts[i])
# print('Translation:', translation)

#input = "read n"
translation = inference_translater(input_seq)
print('Input:', input_texts[i])
print('Translation:', translation)

Input: create integer n
Translation: int main() {



In [108]:
encoder_input_data[7820]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)